## Example: Online Planning in the Lava Grid World - Forward Tree Search

This example will familiarize students with the `rollout` solution of a `two-dimensional` navigation problem, i.e., the lava world [roomba](https://www.irobot.com) problem we have discussed. 

### Problem
You have a [roomba](https://www.irobot.com) that has finished cleaning the kitchen floor and needs to return to its charging station. However, between your kitchen floor and the `charging station` (safety), there are one or more `lava pits` (destruction for the [roomba](https://www.irobot.com)). This is an example of a two-dimensional grid-world navigational decision task. 

This example will familiarize students with using `forward tree search` for solving a two-dimensional grid-world navigation task, the role of the discount factor $\gamma$. In particular, we will:

* __Task 1__: Build a `5` $\times$ `5` world model with two lava pits and a charging station.
* __Task 2__: Generate the components of the MDP problem 
* __Task 3__: Develop on online planning solution by implemeting the `forward_search(...)` routine of `Algorithm 9.2` of the [Decisions Book](https://algorithmsbook.com)

In [1]:
include("Include.jl");

    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
   Resolving package versions...
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`
  Activating project at `~/Desktop/julia_work/CHEME-5760-Examples-F23`
    Updating registry at `~/.julia/registries/General.toml`
    Updating git-repo `https://github.com/varnerlab/VLDecisionsPackage.jl.git`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Project.toml`
  No Changes to `~/Desktop/julia_work/CHEME-5760-Examples-F23/Manifest.toml`


## Task 1: Build the world model
We encoded the `rectangular grid world` using the `MyRectangularGridWorldModel` model, which we construct using a `build(...)` method. Let's setup the data for the world, setup the states, actions, rewards and then construct the world model. 
* First, set values for the `number_of_rows` and `number_of_cols` variables, the `nactions` that are avialble to the agent and the `discount factor` $\gamma$. 
* Then, we'll compute the number of states, and setup the state set $\mathcal{S}$ and the action set $\mathcal{A}$

In [2]:
number_of_rows = 5
number_of_cols = 5
nactions = 4;
γ = 0.20;
nstates = (number_of_rows*number_of_cols);
𝒮 = range(1,stop=nstates,step=1) |> collect;
𝒜 = range(1,stop=nactions,step=1) |> collect;

Next, we'll set up a description of the rewards, the `rewards::Dict{Tuple{Int,Int}, Float64}` dictionary, which maps the $(x,y)$-coordinates to a reward value. We only need to put `non-default` reward values in the reward dictionary (we'll add default values to the other locations later). Lastly, let's put the locations on the grid that are `absorbing`, meaning the charging station or lava pits in your living room:

In [3]:
# setup rewards -
rewards = Dict{Tuple{Int,Int}, Float64}()
rewards[(2,2)] = -100000.0 # lava in the (2,2) square 
rewards[(4,4)] = -100000.0 # lava in the (4,4) square
rewards[(3,3)] = 1000.0    # charging station square

# setup set of absorbing states -
absorbing_state_set = Set{Tuple{Int,Int}}()
push!(absorbing_state_set, (2,2));
push!(absorbing_state_set, (3,3));
push!(absorbing_state_set, (4,4));

Finally, we can build an instance of the `MyRectangularGridWorldModel` type, which models the grid world. We save this instance in the `world` variable
* We must pass in the number of rows `nrows`, number of cols `ncols`, and our initial reward description in the `rewards` field into the `build(...)` method

In [4]:
world = VLDecisionsPackage.build(MyRectangularGridWorldModel, 
    (nrows = number_of_rows, ncols = number_of_cols, rewards = rewards));

## Task 2: Generate the components of the MDP problem
The MDP problem requires the return function (or array) `R(s, a)`, and the transition function (or array) `T(s, s′, a)`. Let's construct these from our grid world model instance, starting with the reward function `R(s, a)`:

### Rewards $R(s,a)$
We'll encode the reward function as a $\dim\mathcal{S}\times\dim\mathcal{A}$ array, which holds the reward values for being in state $s\in\mathcal{S}$ and taking action $a\in\mathcal{A}$. After initializing the `R`-array and filling it with zeros, we'll populate the non-zero values of $R(s, a)$ using nested `for` loops. During each iteration of the `outer` loop, we'll:
* Select a state `s`, an action `a`, and a move `Δ`
* We'll then compute the new position resulting from implementing action `a` from the current position and store this in the `new_position` variable. * If the `new_position`$\in\mathcal{S}$ is in our initial `rewards` dictionary (the charging station or a lava pit), we use that reward value from the `rewards` dictionary. If we are still in the world but not in a special location, we set the reward to `-1`.
* Finally, if `new_position`$\notin\mathcal{S}$, i.e., the `new_position` is a space outside the grid, we set a penalty of `-50000.0`.

In [5]:
R = zeros(nstates, nactions);
fill!(R, 0.0)
for s ∈ 𝒮
    for a ∈ 𝒜
        
        Δ = world.moves[a];
        current_position = world.coordinates[s]
        new_position =  current_position .+ Δ
        if (haskey(world.states, new_position) == true)
            if (haskey(rewards, new_position) == true)
                R[s,a] = rewards[new_position];
            else
                R[s,a] = -1.0;
            end
        else
            R[s,a] = -50000.0; # we are off the grid, big negative penalty
        end
    end
end
R

25×4 Matrix{Float64}:
  -50000.0       -1.0   -50000.0       -1.0
  -50000.0  -100000.0       -1.0       -1.0
  -50000.0       -1.0       -1.0       -1.0
  -50000.0       -1.0       -1.0       -1.0
  -50000.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0  -100000.0
      -1.0       -1.0       -1.0       -1.0
      -1.0     1000.0  -100000.0       -1.0
      -1.0       -1.0       -1.0       -1.0
      -1.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0       -1.0
 -100000.0       -1.0       -1.0     1000.0
      -1.0       -1.0       -1.0       -1.0
      -1.0  -100000.0     1000.0       -1.0
      -1.0       -1.0       -1.0   -50000.0
      -1.0       -1.0   -50000.0       -1.0
      -1.0       -1.0       -1.0       -1.0
    1000.0       -1.0       -1.0  -100000.0
      -1.0       -1.0       -1.0       -1.0
      -1.0       -1.0  -100000.0   -50000.0
      -1.0   -50000.0   -50000.0       -1.0
      -1.0   -50000.0       -1.0       -1.0
      -1.0

### Transition $T(s, s^{\prime},a)$
Next, build the transition function $T(s,s^{\prime},a)$. We'll encode this as a $\dim\mathcal{S}\times\dim\mathcal{S}\times\dim\mathcal{A}$ [multidimension array](https://docs.julialang.org/en/v1/manual/arrays/) and populate it using nested `for` loops. 

* The `outer` loop we will iterate over actions. For every $a\in\mathcal{A}$ will get the move associated with that action and store it in the `Δ`
* In the `inner` loop, we will iterate over states $s\in\mathcal{S}$. We compute a `new_position` resulting from implementing action $a$ and check if `new_position`$\in\mathcal{S}$. If `new_position` is in the world, and `current_position` is _not_ an `absorbing state` we set $s^{\prime}\leftarrow$`world.states[new_position]`, and `T[s, s′,  a] = 1.0`
* However, if the `new_position` is outside of the grid (or we are jumping from an `absorbing` state), we set `T[s, s,  a] = 1.0`, i.e., the probability that we stay in `s` if we take action `a` is `1.0`.

In [6]:
T = Array{Float64,3}(undef, nstates, nstates, nactions);
fill!(T, 0.0)
for a ∈ 𝒜
    
    Δ = world.moves[a];
    
    for s ∈ 𝒮
        current_position = world.coordinates[s]
        new_position =  current_position .+ Δ
        if (haskey(world.states, new_position) == true && 
                in(current_position, absorbing_state_set) == false)
            s′ = world.states[new_position];
            T[s, s′,  a] = 1.0
        else
            T[s, s,  a] = 1.0
        end
    end
end

Finally, we construct an instance of the `MyMDPProblemModel` which encodes the data required to solve the MDP problem.
* We must pass the states `𝒮`, the actions `𝒜`, the transition matrix `T`, the reward matrix `R`, and the discount factor `γ` into the `build(...)` method. We store the MDP model in the `m` variable:

In [7]:
m = VLDecisionsPackage.build(MyMDPProblemModel, 
    (𝒮 = 𝒮, 𝒜 = 𝒜, T = T, R = R, γ = γ));

## Task 3: Online planning solution: Forward Tree Search
First, let's set the `depth` that are going to explore, i.e., how many steps are we going to take when exploring each state `s`:

In [31]:
d = 4;

Next, let's implement the functions:

> The `mylookahead(p::MyMDPProblemModel, U::Function, s::Int64, a::Int64) -> Float64` function computes the immediate reward for taking action `a` in state `s`, and then the expected future utility using the state transition model $T(s,s^{\prime},a)$ and the value (utility) function $U(s)$

> The `forward_search(problem::MyMDPProblemModel,  s::Int64,  d::Int64, U::Function) -> NamedTuple` function computes the best action `a` starting at state `s` using a `forward search tree` method, where we explore the tree using a `depth-first` approach to a depth `d` 

These implementations were based on `Algorithm 9.12` of the [Decisions Book](https://algorithmsbook.com)

In [32]:
function mylookahead(p::MyMDPProblemModel, U::Function, s::Int64, a::Int64)::Float64
    
    # println("mylookahead:starting ... with  s=$(s), a=$(a)");
    
    # get data from the problem -
    𝒮, T, R, γ = p.𝒮, p.T, p.R, p.γ;
    tmp = R[s,a] + γ*sum(T[s, s′,a]*U(s′) for s′ in 𝒮);
    # println("mylookahead: finshed mylookup: s=$(s), a=$(a), value=$(tmp)");
    
    return tmp;
end

mylookahead (generic function with 1 method)

In [33]:
function forward_search(problem::MyMDPProblemModel,  s::Int64,  d::Int64, U::Function)::NamedTuple
    
    if (d ≤ 0)
        # println("forward_search: base condition L4: d = $(d), s=$(s) and u=$(U(s))")
        return (a=nothing, u=U(s))
    end
    best = (a=nothing, u=-Inf)
    
    # println("forward_search: main recursion L9 ... s=$(s), d=$(d) -> d=$(d-1)")
    U′(s) = forward_search(problem, s, d-1, U).u
    # println("forward_search: starting to test actions L11: s=$(s), d=$(d) and U′(s) = $(U′(s))")
    for a ∈ problem.𝒜
        # println("testing action a = $(a)");
        u = mylookahead(problem, U′, s, a)
        if (u > best.u)
            best = (a=a,u=u)
        end
    end
    
    # println("forward_search: completed action testing L20 with s=$(s), d=$(d): returning best=$(best)")
    return best
end;

In [21]:
# Utest(s) = -1;
# forward_search(m,1,2,Utest)

### Compute the policy $\text{my_}\pi$
Initialize a starting function $U(s)$, and then compute the best action `a` for each starting state `s` by calling the `forward_search(...)` function in a `for` loop iterating over states $s\in\mathcal{S}$:

In [34]:
Uₒ(s) = 0;
my_π = Array{Int64,1}()
for s ∈ 𝒮
    a = forward_search(m, s, d, Uₒ).a
    push!(my_π,a)
end

LoadError: InterruptException:

In [29]:
my_π

25-element Vector{Int64}:
 2
 4
 2
 2
 2
 2
 1
 2
 2
 2
 4
 4
 1
 3
 3
 1
 1
 1
 1
 1
 1
 1
 1
 3
 1

Finally, we can extract the policy $\pi(s)$ from the action-value function $Q(s,a)$ using the `policy(...)` function:

### Visualize

In [14]:
move_arrows = Dict{Int,Any}();
move_arrows[1] = "←"
move_arrows[2] = "→"
move_arrows[3] = "↓"
move_arrows[4] = "↑"
move_arrows[5] = "∅";

In [30]:
for s ∈ 𝒮
    a = my_π[s];
    Δ = world.moves[a];
    current_position = world.coordinates[s]
    new_position =  current_position .+ Δ
    
    if (in(current_position, absorbing_state_set) == true)
        println("$(current_position) $(move_arrows[5])")
    else
        println("$(current_position) $(move_arrows[a]) $(new_position)")
    end
end

(1, 1) → (2, 1)
(1, 2) ↑ (1, 3)
(1, 3) → (2, 3)
(1, 4) → (2, 4)
(1, 5) → (2, 5)
(2, 1) → (3, 1)
(2, 2) ∅
(2, 3) → (3, 3)
(2, 4) → (3, 4)
(2, 5) → (3, 5)
(3, 1) ↑ (3, 2)
(3, 2) ↑ (3, 3)
(3, 3) ∅
(3, 4) ↓ (3, 3)
(3, 5) ↓ (3, 4)
(4, 1) ← (3, 1)
(4, 2) ← (3, 2)
(4, 3) ← (3, 3)
(4, 4) ∅
(4, 5) ← (3, 5)
(5, 1) ← (4, 1)
(5, 2) ← (4, 2)
(5, 3) ← (4, 3)
(5, 4) ↓ (5, 3)
(5, 5) ← (4, 5)
